<a href="https://colab.research.google.com/github/sclaudiobr/algoritmos-py/blob/master/GeoCloud02_Mapas_Tematicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geobr # Base de dados do IBGE, Malahas municipais, biomas, microrregiões, sedes
!pip install shapely # conversor e leitor de arquivos shp 
!pip install OWSLib # Acesso ao banco de dados WFS/WMS/OWS
!pip install pygeos # Biblioteca de apoio ao Geopandas
import numpy as np
import geobr as br
import geopandas as gpd
import os
import owslib.wfs as ows
from owslib.wfs import WebFeatureService
from shapely.geometry import box
import zipfile

os.mkdir("shp")

### ----------------------- FUNÇÂO BAIXAR DADOS -----------------------------
def gera_dados(uf,mun,id):
   df = br.read_municipality(code_muni=uf)
   df = df[df.name_muni==mun]
   #df.to_file('sm.geojson', driver='GeoJSON')
   dfbox = box(*df.total_bounds).bounds
   latitude,longitude = np.array(dfbox)[1],np.array(dfbox)[2]
   wfs_url = 'https://geoservicos.ibge.gov.br/geoserver/wfs'
   # Conectando ao serviço do GeoServer WFS.
   wfs = WebFeatureService(wfs_url, version='2.0.0')
   data = wfs.getfeature(typename=id, bbox=dfbox, outputFormat='JSON')
   fn = 'df.geojson'
   with open(fn, 'wb') as fh:
      fh.write(data.read())
   layers = gpd.read_file(fn)
   recorte = gpd.clip(layers, df,keep_geom_type=False)
   recorte.to_file('shp/df.shp', driver='ESRI Shapefile')
### ----------------- UF, MUNICIPIO, TEMA


# Compacta a pasta com o nome do ID escolhido
def zip(src, dst):
    zf = zipfile.ZipFile("%s.zip" % (dst), "w", zipfile.ZIP_DEFLATED)
    abs_src = os.path.abspath(src)
    for dirname, subdirs, files in os.walk(src):
        for filename in files:
            absname = os.path.abspath(os.path.join(dirname, filename))
            arcname = absname[len(abs_src) + 1:]
            print('zipping %s as %s' % (os.path.join(dirname, filename), arcname))
            zf.write(absname, arcname)
    zf.close()



# Identficadores
geo = 'CREN:Geologia_area_Brasil'
geom = 'CREN:brasil_geom_area'
ped = 'CREN:Pedologia_area_Brasil'
veg = 'CREN:vegetacao_area_brasil'

#### TROCA OS DADOS AQUI
gera_dados('RS','Silveira Martins',veg)
### noe do ZIP
zip("projeto","vegetacao")